# Grad-CAM Comparative Analysis (Baseline vs Fine-Tuned)

Purpose:
Compare Grad-CAM heatmaps from baseline and fine-tuned ResNet-50
models to visualize how fine-tuning shifts attention within X-rays.

Output:
- Overlays saved in reports/week2_gradcam_comparison/
- Summary CSV with activation statistics


In [1]:
# Add script to include project root in sys.path

import sys
from pathlib import Path

# Add project root (one level up from notebooks/)
project_root = Path(__file__).resolve().parent.parent if "__file__" in globals() else Path.cwd().parent
sys.path.append(str(project_root))

In [2]:
# Import necessary libraries

import torch
from torchvision import models, transforms
from src.gradcam import GradCAM
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path